# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [63]:
# Read the weather data and the study results
weather = pd.read_csv("weatherpy.csv")
weather_df = pd.DataFrame(weather)
weather_df.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tuktoyaktuk,69.4541,-133.0374,-2.20,78,75,13.80,CA,1615852949
1,capoterra,39.1763,8.9718,55.99,67,20,19.57,IT,1615852949
2,hithadhoo,-0.6000,73.0833,83.23,69,80,10.09,MV,1615852950
3,prado,-17.3411,-39.2208,76.37,79,0,7.67,BR,1615852951
4,east london,-33.0153,27.9116,69.89,78,83,15.55,ZA,1615852953


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [64]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Long' into  locations 
locations = weather_df[["Lat", "Long"]].astype(float)

#Store Humidity as the weight
hum_rate = weather_df["Humidity"].astype(float)


In [65]:
# Create a poverty Heatmap layer
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=hum_rate, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2, opacity=1)

fig.add_layer(heat_layer)

plt.savefig("vacation_heat_map.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [28]:
#A max temperature lower than 80 degrees but higher than 70.
temp1 = weather_df.loc[weather_df["Max Temp"]<80,:]
temp2 = temp1.loc[weather_df["Max Temp"]>70,:]
temp2.head()

#Wind speed less than 10 mph.
wind = temp2.loc[temp2["Wind Speed"]<10,:]
wind.head()

#Zero cloudiness.
cloud = wind.loc[wind["Cloudiness"]==0,:]
cloud.head()

,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,kavali,14.9167,79.9833,77.27,73,0,9.93,IN,1615736902
11,raghogarh,24.4500,77.2000,79.65,18,0,4.83,IN,1615736906
20,abha,18.2164,42.5053,73.40,52,0,6.91,SA,1615736910
45,chuy,-33.6971,-53.4616,75.70,53,0,9.62,UY,1615736921
68,ibra,22.6906,58.5334,78.55,20,0,8.28,OM,1615736930


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:

#Store into variable named hotel_df.
#Add a "Hotel Name" column to the DataFrame.
hotel_df = pd.DataFrame()
fifth = pd.DataFrame(cloud)
hotel_df['City'] = fifth['City']
hotel_df['Country'] = fifth['Country']
hotel_df['Lat'] = fifth['Lat']
hotel_df['Lng'] = fifth['Long']
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,kavali,IN,14.9167,79.9833,
11,raghogarh,IN,24.4500,77.2000,
20,abha,SA,18.2164,42.5053,
45,chuy,UY,-33.6971,-53.4616,
68,ibra,OM,22.6906,58.5334,


In [46]:
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get restaurant type from df
    latitude = row['Lat']
    longitude = row['Lng']
    city = row['City']
    country = row['Country']

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # extract results
    results = response['results']
    
    try:
        hotelname = response['results'][7]['name']
        print(f"Closest hotel to {city} at {latitude} , {longitude} is {hotelname}.")
        
        hotel_df.loc[index, "Hotel Name"] = hotelname
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: kavali.
Closest hotel to kavali at 14.9167 , 79.9833 is SBI ATM.
------------
Retrieving Results for Index 11: raghogarh.
Closest hotel to raghogarh at 24.45 , 77.2 is Govt Hospital.
------------
Retrieving Results for Index 20: abha.
Closest hotel to abha at 18.2164 , 42.5053 is Abha Elite furnished units.
------------
Retrieving Results for Index 45: chuy.
Closest hotel to chuy at -33.6971 , -53.4616 is Gaboto.
------------
Retrieving Results for Index 68: ibra.
Closest hotel to ibra at 22.6906 , 58.5334 is مواصلات Mwasalt.
------------
Retrieving Results for Index 69: mar del plata.
Closest hotel to mar del plata at -38.0023 , -57.5575 is HOTEL GUERRERO.
------------
Retrieving Results for Index 102: phan thiet.
Closest hotel to phan thiet at 10.9333 , 108.1 is Nhà nghỉ Hải Anh.
------------
Retrieving Results for Index 128: faya.
Closest hotel to faya at 18.3851 , 42.4509 is ابتدائية المأمون.
------------
Retrieving Results for Index 142: marrakesh.


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
location = hotel_df[["Lat", "Lng"]]

In [58]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level=2)


marker_layer = gmaps.marker_layer(location, hover_text='', label='', info_box_content=hotel_info)

# Display figure
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
plt.savefig("vacation_heatmap_hotels.png")

fig




Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>